In [3]:
import pandas as pd
from tqdm import tqdm, tqdm_pandas
tqdm.pandas()

import numpy as np
import re

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
train = pd.read_csv('../data/stk_hld_train.csv')
hist = pd.read_csv('../data/stk_bnc_hist.csv')
cus = pd.read_csv('../data/cus_info.csv')
iem = pd.read_csv('../data/iem_info.csv')
test = pd.read_csv('../data/stk_hld_test.csv')

submission = pd.read_csv('../data/sample_submission.csv')

# 개장일 정보
open_day = pd.read_csv('../data/open.csv')

# 주가 정보
stock = pd.read_csv('../data/주가정보_2016_2021.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/stk_hld_train.csv'

In [6]:
# 주가 정보 전처리

# 컬럼을 기존 데이터에 맞게 변경
stock_col = list(stock.columns)
def get_d(day):
    return int(re.sub('[^0-9]+','',day[:10]))
stock_col[1:] = list(map(get_d,stock_col[1:]))
stock.columns = stock_col

# 코드가 int로 바뀌면서 앞의 0이 사라지는 경우
stock['코드'] = stock['코드'].astype('str')
def change(text):
    if len(text)<6:
        tmp = 6-len(text)
        return ('0'*tmp)+text
    else:
        return text

stock['코드'] = stock['코드'].apply(lambda x: change(x))

In [7]:
def get_hist_d(d1,cnt):
    tmp = open_day[open_day['0']>=d1]
    tmp = tmp.iloc[cnt-1].values
    #print(tmp)
    if (int(tmp[0]/10000) - int(d1/10000))>=1:
        y = int(tmp[0]/10000)*10000 + 101
        tmp = open_day[d1<=open_day['0']]
        tmp = tmp[tmp['0'] < y]
        return len(tmp)
    else:
        return int(cnt*0.6)
    return tmp[0]

train['hist_d'] = train.progress_apply(lambda x : get_hist_d(x['byn_dt'],x['hold_d']),axis=1)
#print(get_hist_d(20200522,155))

100%|████████████████████████████████████████████████████████████████████████| 681472/681472 [03:10<00:00, 3573.22it/s]


In [8]:
def get_sell_d(d1,cnt):
    tmp = open_day[open_day['0']>=d1] # d1 이후
    tmp = tmp.iloc[cnt-1].values
    return tmp[-1]

def get_yield(iem_cd,d1,d2):
    iem_stock = stock[stock['코드']==iem_cd[1:]].reset_index(drop=True)
    if len(iem_stock) == 0: # 주가 정보 없는 경우
        #print(iem_cd)
        return 0
    d1_stock = iem_stock[d1].iloc[3]
    d2_stock = iem_stock[d2].iloc[3]
    return d2_stock/d1_stock

train['sell_d'] = train.progress_apply(lambda x : get_sell_d(x['byn_dt'],x['hold_d']),axis=1)
train['yield'] = train.progress_apply(lambda x: get_yield(x['iem_cd'],x['byn_dt'],x['sell_d']),axis=1)
train.drop(['sell_d'], axis = 1, inplace = True)

# test['sell_d'] = test.progress_apply(lambda x : get_sell_d(x['byn_dt'],x['hist_d']),axis=1)
# test['yield'] = test.progress_apply(lambda x: get_yield(x['iem_cd'],x['byn_dt'],x['sell_d']),axis=1)
# test.drop(['sell_d'], axis = 1, inplace = True)

100%|█████████████████████████████████████████████████████████████████████████| 681472/681472 [11:34<00:00, 980.77it/s]


In [10]:
def get_yield(act_id):
    try:
        return yield_dict[act_id]
    except:
        return 1

yield_dict = train.groupby('act_id')['yield'].mean().to_dict()
test['yield'] = test.progress_apply(lambda x: get_yield(x['act_id']),axis=1)

100%|████████████████████████████████████████████████████████████████████████| 70596/70596 [00:00<00:00, 160962.91it/s]
